In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from os import listdir
import re

In [ ]:
df=pd.read_csv('soft_pf0.1_sp100_lp100_reduced10_filament.orientation_corr',skiprows=1,delim_whitespace=True)

In [ ]:
df['orientation_corr_var'] = df['orientation_corr_sem'].apply(lambda y: y**2)

In [ ]:
plt.figure()
df.plot(x='time',y='orientation_corr_avg',yerr='orientation_corr_sem',kind='scatter')
plt.show()

In [ ]:
gb=df.groupby('time')
x=df.time.unique()
y=gb.orientation_corr_avg.mean()
ysem=np.sqrt(gb.orientation_corr_var.mean())
model=sm.GLM(y, x, family=sm.families.Gaussian(sm.families.links.log))
fit=model.fit()
tau=-1.0/fit.params.x1
print("tau = %2.5f" % tau)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
ax.errorbar(x,y,yerr=ysem)
theory = lambda t: np.exp(-t/tau)
ax.plot(x,theory(x),'r')
ax.set_ylabel(r"$\langle u(0)\cdot u(t)\rangle$",fontsize=18)
ax.set_xlabel("Time (t)",fontsize=18)
ax.legend([r"Fit, $\tau=%2.1f$" % tau,'Simulation'],fontsize=16)
ax.grid(True,linestyle='--')

In [ ]:
df=pd.read_csv('soft_pf0.1_sp100_lp100_reduced10_filament.orientation_corr',skiprows=1,delim_whitespace=True)

In [ ]:
dirName='.'
fileNames = [dirName + '/'+i for i in listdir(dirName) if i[-17:] == '.orientation_corr']

if len(fileNames)==0:
    print("No '.orientation_corr' files found in directory '"+dirName+"'.")

D = {"file_name" : []}
for index,name in enumerate(fileNames):
    f = open(name,'r')
    for i in range(2): colNames = f.readline().split()
    if index == 0:
        for colName in colNames:
            D[colName] = []
    D["file_name"].append(name)
    simParams = f.readline().split()
    for colIndex,colName in enumerate(colNames):
        D[colName].append(float(simParams[colIndex]))
    f.close()
#df = pd.DataFrame(D)

In [ ]:
fileNames

In [ ]:
s=set()
for i in fileNames:
    matchObj = re.search('soft_pf0.\d{1,2}_sp\d{3}_lp\d{3}', i)
    #matchObj = re.search('soft_pf0.\d{1,2}_l\d{3}_d\d{3}_sp\d{3}_lp\d{3}',i)
    if (matchObj != None):
        s.add(matchObj.group())
if (len(s) == 0):
    print('No match found')

In [ ]:
dfs = {}
for i in s:
    dfs[i] = None
    for j in fileNames:
        if (j[2:len(i)+2]==i):
            if (type(dfs[i]) == type(None)):
                dfs[i] = pd.read_csv(j,skiprows=1,delim_whitespace=True)
            else:
                dfs[i] = dfs[i].append(pd.read_csv(j,skiprows=1,delim_whitespace=True),ignore_index=True)

In [ ]:
for fname,df in zip(dfs.keys(),dfs.values()):
    if('pf0.1' in fname):
        continue
    df['orientation_corr_var'] = df['orientation_corr_sem'].apply(lambda y: y**2)
    gb=df.groupby('time')
    x=df.time.unique()
    y=gb.orientation_corr_avg.mean()
    ysem=np.sqrt(gb.orientation_corr_var.mean())
    model=sm.GLM(y, x, family=sm.families.Gaussian(sm.families.links.log))
    fit=model.fit()
    tau=-1.0/fit.params.x1
    print("tau = %2.5f" % tau)
    fig,ax = plt.subplots(1,1,figsize=(8,6))
    ax.errorbar(x,y,yerr=ysem)
    theory = lambda t: np.exp(-t/tau)
    ax.plot(x,theory(x),'r')
    ax.set_ylabel(r"$\langle u(0)\cdot u(t)\rangle$",fontsize=18)
    ax.set_xlabel("Time (t)",fontsize=18)
    ax.legend([r"Fit, $\tau=%2.2f$" % tau,'Simulation'],fontsize=16)
    ax.grid(True,linestyle='--')
    ax.set_title(fname,fontsize=20)

In [ ]:
for df in dfs:
    print(df)

In [ ]:
dfs.values()